<a href="https://colab.research.google.com/github/coolcmyk/mitsuki/blob/main/mitsukiConcept_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q datasets
!huggingface-cli login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: w

In [ ]:
from datasets import load_dataset
import re

# Load the dataset
dataset = load_dataset('timdettmers/openassistant-guanaco')

# Shuffle the dataset and slice it
dataset = dataset['train'].shuffle(seed=42).select(range(1000))

def transform_conversation(example):
    # Define Mitsuki's details
    mitsuki_full_name = "Mitsuki Suzuki"
    mitsuki_age = "25 years old"
    mitsuki_personality = "funny and cheerful"
    mitsuki_background = "from the bustling city of Tokyo"

    conversation_text = example['text']
    segments = conversation_text.split('###')

    reformatted_segments = []

    # Iterate over pairs of segments
    for i in range(1, len(segments) - 1, 2):
        human_text = segments[i].strip().replace('Human:', '').strip()

        # Check if there is a corresponding assistant segment before processing
        if i + 1 < len(segments):
            assistant_text = segments[i+1].strip().replace('Assistant:', '').strip()

            # Apply the Mitsuki instruction template with details
            reformatted_segments.append(f'<s>[INST] Acting as {mitsuki_full_name}, {mitsuki_age}, '
                                       f'{mitsuki_personality} personality, '
                                       f'originating from {mitsuki_background}: {assistant_text} [/INST] </s>')
        else:
            # Handle the case where there is no corresponding assistant segment
            reformatted_segments.append(f'<s>[INST] Acting as {mitsuki_full_name}, {mitsuki_age}, '
                                       f'{mitsuki_personality} personality, '
                                       f'originating from {mitsuki_background}: {human_text} [/INST] </s>')

    return {'text': ''.join(reformatted_segments)}


# Apply the transformation
transformed_dataset = dataset.map(transform_conversation)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
transformed_dataset.push_to_hub("kyomoyo/mitsuki-1k")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/kyomoyo/mitsuki-1k/commit/2b09c7bea40b8865e52ec464f3d762219c7ab210', commit_message='Upload dataset', commit_description='', oid='2b09c7bea40b8865e52ec464f3d762219c7ab210', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
#RAW TRAINING WITHOUT ANY OPTIMIZATION



!pip install datasets
from datasets import load_dataset# Assuming you have loaded your Mitsuki dataset using load_dataset("kyomoyo/mitsuki-1k")
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# Assuming you have loaded your Mitsuki dataset using load_dataset("kyomoyo/mitsuki-1k")
mitsuki_dataset = load_dataset("kyomoyo/mitsuki-1k")

# Access the training split (assuming it's the default split name 'train')
mitsuki_train_dataset = mitsuki_dataset['train']

# Sample a conversation from the Mitsuki dataset
sample_conversation = mitsuki_train_dataset[0]['text']

# Load the pre-trained model and tokenizer
model_name = "NousResearch/Llama-2-7b-chat-hf"  # Replace with the actual model name
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Run text generation pipeline with the model
prompt = f"<s>[INST] {sample_conversation} [/INST]"
text_generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = text_generator(prompt)

# Print the generated text
print(result[0]['generated_text'])


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]